In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import networkx as nx
import heapq
import time
from utils_MIP import GraphConstructionDiscretization, biobjective_search, extract_costs, check_pareto_optimality, plot_costs, reconstruct_solution_paths
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# =============================================================================
# Constants, UAV parameters, and initial conditions
# =============================================================================
# Define QZ circles as tuples (x, y, radius, only_electric_radius, risk_limit, toggle_only_electric_or_no_path) toggle_only_electric_or_no_path = 0 for no path, 1 for only electric
map_qz = [(0.0, 0.0, 6.0, 30), (12.0, 10.0, 4.0, 30)]

max_risk_limit = 2/3*sum( [circle[-2] for circle in map_qz] )
acceptable_risk_limit = max_risk_limit/2
start = (-5, -5)       # Starting point
goal = (15, 15)        # Goal point

# UAV characteristics
alpha = 0.5
recharge_factor = 2
beta = alpha / recharge_factor

q_min, q_max, q_start = 20, 100, 100
discretization_angle = 10

In [ ]:
# =============================================================================
# Graph Construction and Node Creation
# =============================================================================

# Instantiate the graph construction object.
# (Assuming you have already refactored your class into GraphConstructionDiscretization.)
graph_object = GraphConstructionDiscretization(
    map_qz, start, goal, q_min, q_max, q_start, alpha, beta, discretization_angle
)

start_time = time.time()
# Create nodes and the index maps.
nodes, index_map, reverse_index_map = graph_object.create_nodes()

# Build the visibility graph
graph_object.build_visibility_graph(reverse_index_map)
end_time = time.time()

In [ ]:
### MIP
model = gp.Model("Uncertain_qz")

### Edges present in the visibility graph
visibility_graph_all_edges = list(graph_object.visibility_graph.edges())
visibility_graph_in_edges, visibility_graph_out_edges = [], []

for u, v, data in graph_object.visibility_graph.edges(data=True):
    edge_type = data['edge_type']
    if edge_type == "internal":
        visibility_graph_in_edges.append((u, v))
    else:
        visibility_graph_out_edges.append((u, v))

### Create variables
a = model.addVars(visibility_graph_all_edges, name="a")
a_t = model.addVars(visibility_graph_all_edges, name="a_t")

### Constraint b is only added for out edges, as in other cases it is zero
b = model.addVars(visibility_graph_all_edges, lb=0, name="b")

### Edge length
edge_length = {}

y = model.addVars(visibility_graph_all_edges, vtype=gp.GRB.BINARY, name="y")
    
### Objective function
model.setObjective( gp.quicksum(b[i, j] for i, j in visibility_graph_all_edges), GRB.MINIMIZE)

### SOC constraints on the edges
for i, j, data in graph_object.visibility_graph.edges(data=True):
    ### Extracting the endpoints of the sides or goal or start
    ### i in this implies initial side and f implies final side, 1 and 2 the endpoints of that sides
    L = sp.sqrt((data["node_i_info"][0] - data["node_j_info"][0])**2 + (data["node_i_info"][1] - data["node_j_info"][1])**2)
    model.addConstr( a[i,j] >=  q_min*y[i,j], f"a_constraints_max_{i}" )
    model.addConstr( a[i,j] <= q_max*y[i,j], f"a_constraints_min_{i}" )
    model.addConstr( a_t[i,j] >= q_min*y[i,j], f"a_t_constraints_max_{j}" )
    model.addConstr( a_t[i,j] <= q_max*y[i,j], f"a_t_constraints_min_{j}" )

    ### SOC constraints when edge is outside quiet zone
    if (i,j) in visibility_graph_out_edges:
        model.addConstr( a[i,j] - a_t[i,j] + (alpha+beta)*b[i,j] - alpha*L*y[i,j] == 0, f"SOC_constraint_outside_quiet_zone_{i}{j}")
        # model.addConstr( b[i,j] <= data["length"]*y[i,j], f"Length_min_constraint_{i}{j}")
        model.addConstr( b[i,j]<= L, f"Length_min_constraint_{i}{j}")

    ### SOC constraints when edge is inside quiet zone
    elif (i,j) in visibility_graph_in_edges:
        only_electric_length = data["only_electric_length"]
        model.addConstr( a[i,j] - a_t[i,j] + (alpha+beta)*b[i,j] - alpha*(L)*y[i,j] == 0 , f"SOC_constraint_inside_quiet_zone_{i}{j}")
        model.addConstr( b[i,j]<= L-only_electric_length, f"Length_min_constraint_{i}{j}")

model.addConstr( gp.quicksum(y[i, j] for i, j in visibility_graph_out_edges if i == "s") == 1 , f"Source_SOC_constraint")
model.addConstr( gp.quicksum(a[i, j] for i, j in visibility_graph_out_edges if i == 0) == q_start , f"Source_SOC_constraint")
### SOC constraint for sink
model.addConstr( gp.quicksum(y[i, j] for i, j in visibility_graph_out_edges if j == "g") == 1 , f"Source_SOC_constraint")
model.addConstr( gp.quicksum(a_t[i, "g"] for i in graph_object.visibility_graph.predecessors("g")) <= q_max , f"Sink_max_SOC_constraint")
model.addConstr( q_min <= gp.quicksum(a_t[i, "g"] for i in graph_object.visibility_graph.predecessors("g")) , f"Sink_min_SOC_constraint")

### Flow constraints for the target nodes along with the source node for y
### Flow out constraints
model.addConstrs( gp.quicksum(y[i,j] for j in graph_object.visibility_graph.successors(i)) == 1 for i in graph_object.visibility_graph.nodes if i == 0)
### Flow in constraints
model.addConstrs( gp.quicksum(y[i,j] for i in graph_object.visibility_graph.predecessors(j)) == 1 for j in graph_object.visibility_graph.nodes if j == "g")
### Flow constraints for all nodes
model.addConstrs( gp.quicksum(y[i,j] for j in graph_object.visibility_graph.successors(i)) == gp.quicksum(y[j,i] for j in graph_object.visibility_graph.predecessors(i)) for i in graph_object.visibility_graph.nodes if i != 0 and i != "g")

model.setParam('OutputFlag', 1)
# model.setParam("NonConvex", )
# model.setParam('MIPGap', model_params["MIPgap"])
# model.setParam("Timelimit", model_params["TimeLimit"])
# model.setParam("OutputFlag", model_params["OutputFlag"])
# model.setParam("PSDTol", model_params["PSDTol"])
model.optimize()


Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-20
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 17289 rows, 11472 columns and 43034 nonzeros
Model fingerprint: 0x98b1bc6a
Variable types: 8604 continuous, 2868 integer (2868 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-15, 1e+02]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
